In [1]:
# Importing dependancies and data from weather_py output csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pygeocoder import Geocoder

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

csv_path = "../weather_py/output_data/weather_csv"

weather_data = pd.read_csv(csv_path)
weather_data.dropna()

,city,country,lat,lng,max_temp (f),humidity (%),cloudiness (%),wind_speed (mph),date
0,bambous virieux,mu,-20.3428,57.7575,77.000,88,75,7.16,2021-03-01
1,albany,au,42.6001,-73.9662,39.002,59,100,1.79,2021-03-01
2,yellowknife,ca,62.4560,-114.3525,-4.000,49,90,4.63,2021-03-01
3,roma,au,43.2128,-75.4557,33.998,60,90,11.32,2021-03-01
4,kahului,us,20.8947,-156.4700,73.400,68,40,8.75,2021-03-01
...,...,...,...,...,...,...,...,...,...
563,lazaro cardenas,mx,17.9583,-102.2000,81.806,57,0,4.22,2021-03-01
564,pacifica,us,37.6138,-122.4869,60.998,31,1,4.63,2021-03-01
565,taltal,cl,-25.4000,-70.4833,72.176,62,29,3.22,2021-03-01
566,road town,vg,18.4167,-64.6167,80.600,61,40,5.14,2021-03-01


In [2]:
location = weather_data[["lat", "lng"]].astype(float)
humidity = weather_data["humidity (%)"].astype(float)

In [3]:
# create gmaps heatmap layer with points in each city of the weather datframe weighted by humidity 
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                    dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Selecting for ideal conditions, personalized to indivual's specifications to create short list of travel destinations
best_cities = weather_data[weather_data["max_temp (f)"] <= 90]
best_cities = best_cities[best_cities["max_temp (f)"] >= 75]
best_cities = best_cities[best_cities["humidity (%)"] <= 80]
best_cities = best_cities[best_cities["humidity (%)"] >= 40]
best_cities = best_cities[best_cities["cloudiness (%)"] <= 20]
best_cities = best_cities[best_cities["wind_speed (mph)"] < 4]
best_cities

,city,country,lat,lng,max_temp (f),humidity (%),cloudiness (%),wind_speed (mph),date
34,yenagoa,ng,4.9247,6.2642,80.978,80,0,3.34,2021-03-01
47,rikitea,pf,-23.1203,-134.9692,78.206,62,0,2.95,2021-03-01
106,tiruchchirappalli,in,10.8167,78.6833,77.000,78,20,2.06,2021-03-01
125,zabid,ye,14.1951,43.3152,79.574,68,0,1.38,2021-03-01
157,salalah,om,17.0151,54.0924,77.000,65,0,1.03,2021-03-01
167,port blair,in,11.6667,92.7500,77.882,63,0,2.43,2021-03-01
198,cap malheureux,mu,-19.9842,57.6142,77.000,79,0,1.34,2021-03-01
238,belmonte,br,-15.8631,-38.8828,86.000,66,20,3.60,2021-03-01
314,lima,pe,-12.0432,-77.0282,82.400,54,0,3.60,2021-03-01
325,kavaratti,in,10.5669,72.6420,81.950,68,0,1.02,2021-03-01


In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting search parameters 
search_type = "hotel"
radius = 5000
hotel_names = []
hotel_lats = []
hotel_lngs = []

params = {
   
    "radius": radius,
    "type": search_type,
    "key": g_key
}

# Iterate through best_cities setting lat/lng as location parameter 
for index, row in best_cities.iterrows():
    
    try:
       
        lat = row["lat"]
        lng = row["lng"]
        city = row["city"]
        
        params["location"] = f"{lat},{lng}"

        search = requests.get(base_url, params=params)
        search_result = search.json()
        
        # Selecting the second hotel from the json object and getting the name and location of the hotel
        hotel_name = search_result["results"][1]["name"]
        hotel_lat = search_result["results"][1]["geometry"]["location"]["lat"] 
        hotel_lng = search_result["results"][1]["geometry"]["location"]["lng"]
        
        # Append to list
        hotel_names.append(hotel_name)
        hotel_lats.append(hotel_lat) 
        hotel_lngs.append(hotel_lng)
        
        print(f"found {hotel_name} in {city}")
              
    except:
              
        print("oops. no hotel found") 

found Ecobank - Mbiama Road Branch in yenagoa
found Pension Maro'i in rikitea
found Sangam Hotel in tiruchchirappalli
found مدرسة أبي موسى الأشعري in zabid
found HAMDAN PLAZA HOTEL SALALAH in salalah
found Welcomhotel Bay Island Port Blair in port blair
found Pereybere Hotel & Spa in cap malheureux
found Belmonte Lighthouse in belmonte
found Gran Hotel Bolivar Lima in lima
found All India Radio Kavaratti in kavaratti
found Municipalidad Distrital de Quilmaná in quilmana
found Hondaafushi Island Resort in kulhudhuffushi
found Carlcare Service in sakassou
found Pension Oaoa in tautira
found Veranda Paul et Virginie Hotel & Spa in grand gaube
found Gahabura Centre de Santé in cibitoke
found Religiosas Misioneras Siervas del Espíritu Santo in niquero
found HS Hotsson Smart Acapulco in acapulco


In [6]:
# Adding hotel information to best_cities
best_cities["hotel"] = hotel_names
best_cities["hotel_lat"] = hotel_lats
best_cities["hotel_lng"] = hotel_lngs
best_cities

,city,country,lat,lng,max_temp (f),humidity (%),cloudiness (%),wind_speed (mph),date,hotel,hotel_lat,hotel_lng
34,yenagoa,ng,4.9247,6.2642,80.978,80,0,3.34,2021-03-01,Ecobank - Mbiama Road Branch,4.925975,6.259329
47,rikitea,pf,-23.1203,-134.9692,78.206,62,0,2.95,2021-03-01,Pension Maro'i,-23.120620,-134.968753
106,tiruchchirappalli,in,10.8167,78.6833,77.000,78,20,2.06,2021-03-01,Sangam Hotel,10.804300,78.676672
125,zabid,ye,14.1951,43.3152,79.574,68,0,1.38,2021-03-01,مدرسة أبي موسى الأشعري,14.201043,43.317534
157,salalah,om,17.0151,54.0924,77.000,65,0,1.03,2021-03-01,HAMDAN PLAZA HOTEL SALALAH,17.019113,54.061124
167,port blair,in,11.6667,92.7500,77.882,63,0,2.43,2021-03-01,Welcomhotel Bay Island Port Blair,11.676177,92.740200
198,cap malheureux,mu,-19.9842,57.6142,77.000,79,0,1.34,2021-03-01,Pereybere Hotel & Spa,-19.994261,57.591778
238,belmonte,br,-15.8631,-38.8828,86.000,66,20,3.60,2021-03-01,Belmonte Lighthouse,-15.863519,-38.876870
314,lima,pe,-12.0432,-77.0282,82.400,54,0,3.60,2021-03-01,Gran Hotel Bolivar Lima,-12.050961,-77.035140
325,kavaratti,in,10.5669,72.6420,81.950,68,0,1.02,2021-03-01,All India Radio Kavaratti,10.557603,72.633873


In [7]:
# Adding the hotel marker layer to the heatmap with pins
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in best_cities.iterrows()]
locations = best_cities[["hotel_lat", "hotel_lng"]]

markers = gmaps.marker_layer(locations)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))